# <font color = "blue"> Workflows in Python</font>
This is to follow the example code from [Katie Malone's blog post](https://civisanalytics.com/blog/data-science/2015/12/17/workflows-in-python-getting-data-ready-to-build-models/) at Civis Analytics. This gives an example of a workflow model for Python. She describes as 

**"a workflow that focuses on getting a quick-and-dirty model up and running as quickly as possible, and then going back to iterate on the weak points until the model seems to be converging on an answer."**

- Dataset: “Pump it Up: Mining the Water Table” challenge on drivendata.org, which has examples of wells in Africa, their characteristics and whether they are **functional, non-functional, or functional but in need of repair.** 

-  Goal: build a model that will take the characteristics of a well and predict correctly which category that well falls into.

-----


# <font color = "blue"> [Part 1: Getting Data Ready to Build Models](https://civisanalytics.com/blog/data-science/2015/12/17/workflows-in-python-getting-data-ready-to-build-models/) </font>

# Getting started
- read in data
- transform features and labels to make the data amenable to machine learning
- pick a modeling strategy (classification)
- make a train/test split (this was done implicitly when I called cross_val_score)
- evaluate several models for identifying wells that are failed or in danger of failing

# 1. Labels
## A quick print statement on the labels shows that the labels are strings

In [1]:
import pandas as pd
import numpy as np
features_df = pd.DataFrame.from_csv("data/training_set_values.csv")
labels_df   = pd.DataFrame.from_csv("data/training_set_labels.csv") 
print(labels_df.head(20))

                  status_group
id                            
69572               functional
8776                functional
34310               functional
67743           non functional
19728               functional
9944                functional
19816           non functional
54551           non functional
53934           non functional
46144               functional
49056               functional
50409               functional
36957               functional
50495               functional
53752               functional
61848               functional
48451           non functional
58155           non functional
34169  functional needs repair
18274               functional


## Mapping labels to integers
"When I want a specific mapping between strings and integers, like here, doing it manually is usually the way I go."
- there’s also the sklearn LabelEncoder.
- pandas applymap()
    - apply() vs. applymap(): applymap() operates on a whole dataframe while apply() operates on a series

### Pandas applymap

In [2]:
def label_map(y):
    if y=="functional":
        return 2
    elif y=="functional needs repair":
        return 1
    else:
        return 0
labels_df = labels_df.applymap(label_map)
print(labels_df.head())

       status_group
id                 
69572             2
8776              2
34310             2
67743             0
19728             2


# 2. Features

In [3]:
print(features_df.head())

       amount_tsh date_recorded        funder  gps_height     installer  \
id                                                                        
69572        6000    2011-03-14         Roman        1390         Roman   
8776            0    2013-03-06       Grumeti        1399       GRUMETI   
34310          25    2013-02-25  Lottery Club         686  World vision   
67743           0    2013-01-28        Unicef         263        UNICEF   
19728           0    2011-07-13   Action In A           0       Artisan   

       longitude   latitude              wpt_name  num_private  \
id                                                               
69572  34.938093  -9.856322                  none            0   
8776   34.698766  -2.147466              Zahanati            0   
34310  37.460664  -3.821329           Kwa Mahundi            0   
67743  38.486161 -11.155298  Zahanati Ya Nanyumbu            0   
19728  31.130847  -1.825359               Shuleni            0   

           

## Many of the features are categorical and they need to be transformed to numerical values.
- transform categorical features: OneHotEncoder in sklearn or get_dummies() in pandas.

In [4]:
def transform_feature( df, column_name ):
    
    """ take features_df and the name of a column in that dataframe, 
        and return the same dataframe but 
        with the indicated feature encoded with integers rather than strings"""
    
    unique_values = set( df[column_name].tolist() )
    transformer_dict = {}
    for ii, value in enumerate(unique_values):
        transformer_dict[value] = ii

    def label_map(y):
        return transformer_dict[y]
    df[column_name] = df[column_name].apply( label_map )
    return df

In [5]:
### list of column names indicating which columns to transform; 
### this is just a start!  Use some of the print( labels_df.head() )
### output upstream to help you decide which columns get the
### transformation

names_of_columns_to_transform = ["funder", "installer", "wpt_name", "basin", "subvillage",
                    "region", "lga", "ward", "public_meeting", "recorded_by",
                    "scheme_management", "scheme_name", "permit",
                    "extraction_type", "extraction_type_group",
                    "extraction_type_class",
                    "management", "management_group",
                    "payment", "payment_type",
                    "water_quality", "quality_group", "quantity", "quantity_group",
                    "source", "source_type", "source_class",
                    "waterpoint_type", "waterpoint_type_group"]

for column in names_of_columns_to_transform:
    features_df = transform_feature( features_df, column )
    
print( features_df.head() )
    
### remove the "date_recorded" column--we're not going to make use
### of time-series data today
features_df.drop("date_recorded", axis=1, inplace=True)

print(features_df.columns.values)

       amount_tsh date_recorded  funder  gps_height  installer  longitude  \
id                                                                          
69572        6000    2011-03-14    1539        1390       1749  34.938093   
8776            0    2013-03-06     774        1399        136  34.698766   
34310          25    2013-02-25     906         686       1400  37.460664   
67743           0    2013-01-28     590         263        556  38.486161   
19728           0    2011-07-13    1341           0        537  31.130847   

        latitude  wpt_name  num_private  basin          ...            \
id                                                      ...             
69572  -9.856322     15203            0      5          ...             
8776   -2.147466      5611            0      7          ...             
34310  -3.821329      6138            0      6          ...             
67743 -11.155298      1252            0      1          ...             
19728  -1.825359     2

## prep for sklearn: convert it to numpy.ndarray

In [6]:
X = features_df.as_matrix()
y = labels_df["status_group"].tolist()

# 3. Train and test 
- The cheapest and easiest way to train on one portion of my dataset and test on another, and to get a measure of model quality at the same time, is to use **sklearn.cross_validation.cross_val_score()**
    - Splits my data into three equal portions, trains on two of them, and tests on the third
    - This process repeats three times.

In [7]:
import sklearn.linear_model
import sklearn.cross_validation
clf = sklearn.linear_model.LogisticRegression()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

[ 0.68363636  0.6830303   0.6859596 ]


# 4. Classification or Regression
- I have the choice of **modeling with a classifier** and potentially getting slightly worse performance, 
- or building **a regression but needing to add a post-processing step that turns my continuous (i.e. float) predictions into integer category labels.** 
- I’ve decided to go with the classification approach for this example, but this is a decision made for convenience that I could revisit when improving my model down the road.

# 5. Compare algorithms
- I started with a simple logistic regression above (despite the name, this is a classification algorithm) 
- I’ll compare to a couple of other classifiers, a decision tree classifier and a random forest classifier, to see which one seems to do the best.

In [8]:
import sklearn.tree
import sklearn.ensemble

clf = sklearn.tree.DecisionTreeClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

clf = sklearn.ensemble.RandomForestClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

[ 0.73959596  0.73580808  0.73020202]
[ 0.78868687  0.78929293  0.78893939]


-----
# <font color = "blue"> [Part 2: Curating Features and Thinking Scientifically about Algorithms](https://civisanalytics.com/blog/data-science/2015/12/23/workflows-in-python-curating-features-and-thinking-scientifically-about-algorithms/) </font>

# 1. Revisiting numerical encoding of categorical features

- A problem with representing categorical variables as integers is that **integers are ordered**, while categories are not. 
- The standard way to deal with this is to use **dummy variables**; **one-hot encoding** is a very common way of dummying. 

## One-hot encoding
- The categories are expanded over several boolean columns, only one of which is true (hot). 
- the scikit-learn `OneHotEncoder` object or pandas `get_dummies()` 

In [9]:
import sklearn.preprocessing

def hot_encoder(df, column_name):
    
    """
        a one-hot-encoder function 
        that takes the data frame and the title of a column, and 
        returns the same data frame but one-hot encoding performed on the indicated feature
    """
    
    column = df[column_name].tolist()
    column = np.reshape( column, (len(column), 1) )  ### needs to be an N x 1 numpy array
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit( column )
    new_column = enc.transform( column ).toarray()
    column_titles = []
    
    ### making titles for the new columns, and appending them to dataframe
    for ii in range( len(new_column[0]) ):
        this_column_name = column_name+"_"+str(ii)
        df[this_column_name] = new_column[:,ii]
    return df

## One-hot encoding makes the dataset bigger–sometimes a lot bigger. 
You can imagine that this can sometimes get really, really big (imagine a column encoding all the counties in the United States, for example).
- There are some columns in this example that will really blow up the dataset, so I’ll remove them before proceeding with the one-hot encoding.

In [10]:
print(features_df.columns.values)

['amount_tsh' 'funder' 'gps_height' 'installer' 'longitude' 'latitude'
 'wpt_name' 'num_private' 'basin' 'subvillage' 'region' 'region_code'
 'district_code' 'lga' 'ward' 'population' 'public_meeting' 'recorded_by'
 'scheme_management' 'scheme_name' 'permit' 'construction_year'
 'extraction_type' 'extraction_type_group' 'extraction_type_class'
 'management' 'management_group' 'payment' 'payment_type' 'water_quality'
 'quality_group' 'quantity' 'quantity_group' 'source' 'source_type'
 'source_class' 'waterpoint_type' 'waterpoint_type_group']


In [11]:
features_df.drop( "funder", axis=1, inplace=True )
features_df.drop( "installer", axis=1, inplace=True )
features_df.drop( "wpt_name", axis=1, inplace=True )
features_df.drop( "subvillage", axis=1, inplace=True )
features_df.drop( "ward", axis=1, inplace=True )

In [12]:
names_of_columns_to_transform.remove("funder")
names_of_columns_to_transform.remove("installer")
names_of_columns_to_transform.remove("wpt_name")
names_of_columns_to_transform.remove("subvillage")
names_of_columns_to_transform.remove("ward")

In [28]:
for feature in names_of_columns_to_transform:
    features_df = hot_encoder( features_df, feature )

print( features_df.head() )

NameError: name 'df' is not defined

Now we have 3000 features (origianlly we had about 40). 

# 2. Feature selection
Having so many features invites problems with overfitting, slow and memory-intensive training.
- This is a perfect use case for feature selection, which is supported in scikit-learn by e.g. `SelectKBest()`, which will do **univariate feature selection** to get the k features (where k is a number which I have to tell the algorithm). 
- Making a guess, I can ask for the top 100 features, which doesn’t make my performance much worse and speeds things up a lot:

In [15]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=100)

In [21]:
X = features_df.as_matrix()
selected_X = select.fit_transform(X, y)

/Users/Hongsup/anaconda/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:111: UserWarning: Features [ 12 191] are constant.
  UserWarning)


# 3. Back to Machine Learning
In the last post, a **random forest classifier did the best** of all the models I tried, beating a logistic regression (by a lot) and a decision tree classifier (by a slimmer margin). 

Here are a few reasons why:

## Logistic regression
- A logistic regression is **an example of a linear model**, which (unless you make special adaptations, which I’ll detail in a moment) assumes that **the relationship between each of my features and the output class is a linear one**. For example, if one of the features is the depth of a well, a linear model will assume that (all other things being equal) the difference in functionality between a 20-foot-deep well and a 40-foot-deep one will be the same as the difference between 40 feet and 60 feet. This isn’t always a valid assumption. 
    - **One way to address it is to add extra features** like depth squared and the logarithm of depth, which helps a linear model capture nonlinearities, but might not still allow me to get all the nuances of nonlinear relationships.

- A logistic regression also **doesn’t capture interactions between features**, for example that deep wells might be largely functional and wells drilled in rock are largely functional but deep wells in rocky places are largely non-functional. 
    - Again, I can explicitly add interaction terms to the logistic regression, but this **gets unwieldy fast when I have many features.**


## Decision Tree
- A decision tree **can capture interactions and nonlinearities much more naturally than logistic regression**, because of the binary tree structure of the decision tree algorithm itself. 
    - The downside of decision trees is that **they can be harder to interpret or assign uncertainties to their predictions.**

## Random Forest
- A random forest is a collection of decision trees, each of which is trained on **a subset of the rows/columns** of the training data. The randomness in the training set means that the individual trees in a random forest are high-variance, but low-bias, and the final prediction is made by **having each tree classify a given event and then using their predictions as “votes,” with the majority opinion being assigned as the label**. 
- I have the nonlinearities and interactions being **captured by the individual trees**, but ensembling many trees into a random forest tends to **cancel out the biases/shortcomings of any one tree** and I get a stronger predictor overall.
- In [empirical studies](https://www.cs.cornell.edu/~caruana/ctp/ct.papers/caruana.icml06.pdf) of many algorithms being applied to many **supervised learning problems**, random forests often come out on top overall. So **when in doubt, or if I only have the time/resources to try one model**, a random forest is likely to get at or near the peak performance of all the algorithms on the market.
- If it was tricky to interpret or compute errors for a decision tree, a random forest is only going to be worse because there’s now 50-100 decision trees to worry about.

## Random forests have lots of parameters to optimize. 
- How many trees should there be? 
- How does each tree get trained? 
- How many features get used in training each tree? 

There usually aren’t formulaic answers to these questions, and part of the craft of machine learning is **tuning these parameters to get the best performance** that I can out of my model. 
- But with so many parameters, which sometimes interact with each other in complex ways, parameter tuning can be a huge hassle. 
- In the next post, I’ll talk about an extremely powerful pair of tools in scikit-learn, the Pipeline and GridSearchCV, that allow crazy powerful parameter tuning in just a few lines of code.

# [Part 3: Using Pipeline and GridSearchCV for More Compact and Comprehensive Code](https://civisanalytics.com/blog/data-science/2016/01/06/workflows-python-using-pipeline-gridsearchcv-for-compact-code/)

- Now we're left with **a lot of free parameters of the algorithms to tune**, and messing around with the workflow often leads to **spaghetti code** that becomes less and less understandable/easy to experiment with as we go.
- Enter the scikit-learn Pipeline and GridSearchCV objects: two tools that effectively allow us to pour gasoline on  data science fire, tightening up the code and doing parameter scans in just a few lines of code.

# 1. Pipeline
- There are a number of tools that I’ve chained together to get where I am now, like SelectKBest and RandomForestClassifier. After selecting the 100 best features, **the natural next step is to run my random forest again to see if it does a little better with fewer features.** In this case, I have SelectKBest doing selection, with the output of that process going straight into a classifier. 
- **Pipeline packages the transformation step of SelectKBest with the estimation step of RandomForestClassifier into a coherent workflow.**


## Benefit
- It makes code more **readable** (or, if you like, it makes the intent of the code clearer).
- I don’t have to worry about keeping track data during intermediate steps, for example between transforming and estimating.
- It makes it trivial to move ordering of the pipeline pieces, or to swap pieces in and out.
- **It allows you to do `GridSearchCV` on your workflow**

In [23]:
import sklearn.pipeline

select = sklearn.feature_selection.SelectKBest(k=100)
clf = sklearn.ensemble.RandomForestClassifier()

steps = [('feature_selection', select), ('random_forest', clf)]

pipeline = sklearn.pipeline.Pipeline(steps)

X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X, y, test_size=0.33, random_state=42)

### fit your pipeline on X_train and y_train
pipeline.fit( X_train, y_train )

### call pipeline.predict() on your X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )

### test your predictions using sklearn.classification_report()
### classification_report gives more info than cross_val_score
### But we need to split train/test separately (with cross_val_score, you don't have to)
report = sklearn.metrics.classification_report( y_test, y_prediction )

### and print the report
print(report)

             precision    recall  f1-score   support

          0       0.76      0.77      0.77      7458
          1       0.42      0.36      0.39      1425
          2       0.80      0.81      0.81     10719

avg / total       0.76      0.76      0.76     19602



/Users/Hongsup/anaconda/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:111: UserWarning: Features [  12  191  206  211  232  238  273  278  287  291  302  303  307  313  322
  324  325  344  353  359  371  378  383  423  429  432  433  437  440  444
  450  453  479  481  496  509  523  529  533  540  542  546  549  561  566
  597  609  620  648  649  656  684  700  727  735  748  751  761  782  797
  810  812  816  826  829  836  845  847  853  855  856  867  889  894  903
  905  912  944  946  965  975  977 1005 1024 1026 1034 1036 1048 1051 1052
 1053 1070 1077 1096 1106 1118 1120 1139 1145 1146 1157 1158 1161 1172 1174
 1180 1188 1191 1197 1200 1203 1205 1242 1244 1247 1256 1262 1277 1283 1286
 1287 1290 1293 1294 1298 1307 1312 1315 1326 1335 1362 1366 1374 1398 1399
 1402 1405 1418 1423 1425 1426 1429 1432 1441 1456 1482 1494 1500 1511 1522
 1530 1537 1545 1560 1561 1563 1566 1586 1606 1613 1620 1628 1629 1637 1638
 1640 1645 1653 1676 1688 1707 1715

# 2. GridSearchCV

When I decided to select the 100 best features, setting that number to 100 was kind of a hand-wavey decision. Similarly, the RandomForestClassifier that I’m using right now has all its parameters set to their default values, which might not be optimal.

So, a straightforward thing to do now is **to try different values of k** (the number of features being used in the model) and **any RandomForestClassifier parameters I want to tune** (for the sake of concreteness, I’ll play with n_estimators and min_samples_split).

- Trying lots of values for each of these free parameters is tedious
- There can sometimes be interactions between the choices I make in one step and the optimal value for a downstream step. 
- To avoid local optima, I should try all the combinations of parameters, and not just vary them independently. If I want to try 5 different values each for k, n_estimators and min_samples_split, that means 5 x 5 x 5 = 125 different combinations to try. Not something I want to do by hand.

`GridSearchCV` allows me to construct a grid of all the combinations of parameters, tries each combination, and then reports back the best combination/model.

In [25]:
import sklearn.grid_search

parameters = dict(feature_selection__k=[100, 200], 
              random_forest__n_estimators=[50, 100, 200],
              random_forest__min_samples_split=[2, 3, 4, 5, 10])

cv = sklearn.grid_search.GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)
y_predictions = cv.predict(X_test)
report = sklearn.metrics.classification_report( y_test, y_predictions )

In [27]:
print report

             precision    recall  f1-score   support

          0       0.83      0.77      0.80      7458
          1       0.56      0.30      0.39      1425
          2       0.80      0.89      0.84     10719

avg / total       0.79      0.80      0.79     19602



## GridSearchCV seems a little scary at first, because the parameter grid is easy to mess up. 
There’s **a particular convention** being followed in the way that **the parameters are named in the parameters dictionary.**
- I need to have the name of the Pipeline step (e.g. feature_selection, not select; or random_forest, not clf), followed by **two underscores**, followed by the name of the parameter (in sklearn parlance) that I want to vary. 

To put this all together in a painfully simple example:

In [ ]:
clf = RandomForestClassifier()
steps = [("my_classifier", clf)]

### “my_classifier” is the name of the random forest classifier in the steps list; 
### min_samples_split is the associated sklearn parameter that I want to vary
parameters = dict{my_classifier__min_samples_split=[2, 3, 4, 5]}  
pipe = Pipeline(steps)
cv = GridSearchCV( pipe, param_grid = parameters)

Once I’ve got the parameter grid set up properly, the power of GridSearchCV is that it multiplies out all the combinations of parameters and tries each one, making a 3-fold cross-validated model for each combination. Then I can ask for predictions from my GridSearchCV object and it will **automatically return to me the “best” set of predictions** (that is, the predictions from the best model that it tried), or I can explicitly ask for the best model/best parameters using methods associated with GridSearchCV. Of course, trying tons of models can be kind of time-consuming, but the outcome is a much better understanding of how my model performance depends on parameters.

## GridSearchCV can be used as a single object.
I should also mention that I can also use GridSearchCV on just a single object, rather than a full Pipeline. For example, I can optimize SelectKBest or the RandomForestClassifier on their own and that will work just fine. But since there can sometimes be interactions between various steps in the analysis, being able to optimize over the full Pipeline is really useful. It’s also trickier to do, which makes it a good example for teaching. 
- Last, GridSearchCV will **automatically cross validate all steps of the analysis**, such as the feature selection–it’s not just the final algorithm that should be cross-validated, but the upstream transforms as well!

-----
# Tutorial Summary
This brings me to the end of this series, about **end-to-end data analysis in scikit-learn and pandas**. 

My goal in these posts is not to show a perfect analysis, or even one that demonstrates all the steps one might try, but instead to **focus on the process**. 
- If I can get something up and running quickly, even if it’s imperfect, I’m in a much better position to understand later on how much my refinements are indeed improving the analysis. 
- At the same time, there are definitely best practices and tools (like Pipeline and GridSearchCV) that will make my life much easier as my work expands. 
- Having a great set of tools in the python data science stack, and knowing when and how to deploy them, leaves me free to spend my time and energy on the most interesting, important and difficult-to-automate tasks.